In this revised version, we will be using class weights as done in https://www.tensorflow.org/tutorials/structured_data/imbalanced_data we will also do transfer learning as was mentioned in the article

In [92]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.exceptions import DataConversionWarning
import warnings
import pandas as pd
from pywt import wavedec
from pywt import waverec
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import VotingClassifier
from keras.utils import to_categorical
import pywt
import scipy
#from keras.utils import plot_model #plot_model(model, to_file='model.png')
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SVMSMOTE
import pylab as pl
from sklearn.utils import resample
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Input, Add, Flatten, Concatenate, MaxPool1D, Conv1D, Bidirectional, LSTM, Reshape

In [93]:
eeg1 = pd.read_csv("train_eeg1.csv").iloc[:, 1:]
eeg2 = pd.read_csv("train_eeg2.csv").iloc[:, 1:]
emg  = pd.read_csv("train_emg.csv").iloc[:, 1:]
df_y = pd.read_csv("train_labels.csv").iloc[:, 1:]

eeg1_t = pd.read_csv("test_eeg1.csv").iloc[:, 1:]
eeg2_t = pd.read_csv("test_eeg2.csv").iloc[:, 1:]
emg_t = pd.read_csv("test_emg.csv").iloc[:, 1:]

### Create 20 seconds segments by combining 5 of the 4-second length segments

In [94]:
## split them again
eeg1v = eeg1.values
eeg2v = eeg2.values
emgv = emg.values 

In [95]:
print(len(eeg1v)//5)

12960


In [96]:
def reshape_sig(eeg1v, eeg2v, emgv):
    new_len = len(emgv)//5
    eeg1_reshaped = np.array([[0]*2560]*new_len)
    eeg2_reshaped = np.array([[0]*2560]*new_len)
    emg_reshaped = np.array([[0]*2560]*new_len)
    for i in range(new_len):  # 11,042 = 55210/5 minus since end will do m + 1 (may cause out of bounds exc.)
        start = i*5
        end = i*5 + 5
        combined1 = ((eeg1v[start:end, :]).reshape(1, 2560))[0]
        combined2 = ((eeg2v[start:end, :]).reshape(1, 2560))[0]
        combined3 = ((emgv[start:end, :]).reshape(1, 2560))[0]
        eeg1_reshaped[i] = combined1
        eeg2_reshaped[i] = combined2
        emg_reshaped[i] = combined3
    return (eeg1_reshaped, eeg2_reshaped, emg_reshaped)

eeg1_reshaped, eeg2_reshaped, emg_reshaped = reshape_sig(eeg1v, eeg2v, emgv)
print(eeg1_reshaped.shape)
print(eeg2_reshaped.shape)
print(emg_reshaped.shape)

(12960, 2560)
(12960, 2560)
(12960, 2560)


## RMS filter of the EMG signal (applied over a second = 128 samples)

In [97]:
def window_rms(a, window_size):
  a2 = np.power(a,2)
  window = 1.0*np.ones(window_size)
  return np.sqrt(np.convolve(a2, window, 'valid'))/float(window_size)

In [98]:
print(emg_reshaped.shape)
def rms_transform(row):
    res = np.array([])
    end = len(row)
    for i in range(end - 128):
        window = row[i:(i + 128)]
        temp = window_rms(window, 128)
        res = np.append(res, temp)
    return res
rms_vals= np.apply_along_axis(rms_transform, axis = 1, arr = emg_reshaped)

(12960, 2560)


In [99]:
print(rms_vals.shape)

(12960, 2432)


## Combine EEG channels

In [100]:
eegs_comb = np.hstack((eeg1_reshaped, eeg2_reshaped))
print(eegs_comb.shape)
def combine_channels(signals):
    sigs = np.split(signals, 2)
    eeg1 = sigs[0]
    eeg2 = sigs[1]
    return np.array((eeg1, eeg2)).T
    
channels = np.apply_along_axis(combine_channels, axis = 1, arr = eegs_comb)
print(channels.shape)

(12960, 5120)
(12960, 2560, 2)


## CNN

In [101]:
signal_left = Input(shape = (2560, 2), name="left_signal")
def left_branch_model():
    x = Conv1D(filters = 64, kernel_size = 50,  strides = 6, activation='relu')(signal_left)
    x = MaxPool1D(pool_size=8, strides=8)(x)
    x = Dropout(0.5)(x)
    x = Conv1D(filters = 128, kernel_size = 8, strides = 1, activation = 'relu', padding = 'same')(x)
    x = Conv1D(filters = 128, kernel_size = 8, strides = 1, activation = 'relu', padding = 'same')(x)
    x = Conv1D(filters = 128, kernel_size = 8, strides = 1, activation = 'relu', padding = 'same')(x)
    x = MaxPool1D(pool_size=4, strides=4)(x)
    print(x.shape)
    return x #tf.keras.Model(inputs = signal, outputs = x)
lbranch = left_branch_model()

(None, 13, 128)


In [102]:
signal_mid = Input(shape = (2560, 2), name="mid_signal")
def mid_branch_model():
    x = Conv1D(filters = 64, kernel_size = 500,  strides = 50, activation='relu', padding = 'same')(signal_mid) # try with and without padding = 'same'
    x = MaxPool1D(pool_size=8, strides=8)(x)
    x = Dropout(0.5)(x)
    x = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu', padding = 'same')(x)
    x = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu', padding = 'same')(x)
    x = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu', padding = 'same')(x)
    x = MaxPool1D(pool_size=2, strides=2)(x)
    print(x.shape)
    return x #tf.keras.Model(inputs = signal, outputs = x)
mbranch = mid_branch_model()

(None, 3, 128)


In [103]:
signal_right = Input(shape=(2432, 1),  name="right_signal")
def right_branch_model():
    z = Conv1D(filters = 64, kernel_size = 500,  strides = 50, activation='relu', padding = 'same')(signal_right)
    z = MaxPool1D(pool_size=4, strides=4)(z)
    z = Dropout(0.5)(z)
    z = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu', padding = 'same')(z)
    z = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu', padding = 'same')(z)
    z = Conv1D(filters = 128, kernel_size = 6, strides = 1, activation = 'relu', padding = 'same')(z)
    z = MaxPool1D(pool_size=2, strides=2)(z)
    print(z.shape)
    return z #tf.keras.Model(inputs = signal, outputs = z)
rbranch = right_branch_model()

(None, 6, 128)


In [104]:
concat_layer = Concatenate(axis = 1, name = 'concat_axis')([lbranch, mbranch, rbranch])
print(concat_layer.shape)

(None, 22, 128)


In [105]:
pre_trained_fc = Flatten()(concat_layer)
pre_trained_fc = Dense(3, activation = 'softmax')(pre_trained_fc)
pre_trained_model = tf.keras.Model(inputs = [signal_left, signal_mid, signal_right], outputs = pre_trained_fc)
for el in pre_trained_model.layers:
    print(el.name)

left_signal
mid_signal
right_signal
conv1d_36
conv1d_40
conv1d_44
max_pooling1d_18
max_pooling1d_20
max_pooling1d_22
dropout_19
dropout_20
dropout_21
conv1d_37
conv1d_41
conv1d_45
conv1d_38
conv1d_42
conv1d_46
conv1d_39
conv1d_43
conv1d_47
max_pooling1d_19
max_pooling1d_21
max_pooling1d_23
concat_axis
flatten_6
dense_13


In [106]:
# SCORING BEGINS HERE (but is not used until later)
s = Dropout(0.5)(concat_layer)

# Left branch (cannot flatten before since LSTM requires input of form (None, x, y) NOT (None, z))
sLeft = Bidirectional(LSTM(units = 512, return_sequences = True, activation='tanh'))(s) #https://stackoverflow.com/questions/40331510/how-to-stack-multiple-lstm-in-keras
sLeft = Bidirectional(LSTM(units = 512, activation='tanh'))(sLeft)
print(sLeft.shape)

# Right branch
q = Flatten()(s)
q = Dense(units = 1024, activation = 'relu')(q)
print(q.shape)

# combine both 1024-length vectors by adding them
q = Add()([q, sLeft])
print(q.shape)

# now dropout 50% and softmax on 3 output neurons
q = Dropout(0.5)(q)
q = Dense(3, activation = 'softmax')(q)

model = tf.keras.Model(inputs = [signal_left, signal_mid, signal_right], outputs = q)

(None, 1024)
(None, 1024)
(None, 1024)


In [107]:
# one hot encode the labels
y = (df_y.values) - 1
print(y.shape)
y_encoded = to_categorical(y)

(64800, 1)


### Cut off last mouse subject from the data to use for cross validation

In [108]:
len_to_cut = len(channels)//3
print(len_to_cut)

4320


In [109]:
len_to_keep = len(channels)- len_to_cut
train_channels = channels[0:len_to_keep]
cv_channels = channels[len_to_keep:]
train_emg = rms_vals[0:len_to_keep]
cv_emg = rms_vals[len_to_keep:]
print(len(train_channels))
print(len(train_emg))

8640
8640


In [110]:
# now its time to train the pre_trained model, save it, pop the last layer off (the dense one)
# shuffle the data
len_channels = np.arange(0, len(train_channels))
np.random.shuffle(len_channels)
train_sh_emg = train_emg[len_channels]
train_sh_channels = train_channels[len_channels]
print(len(train_sh_emg))
print(len(train_sh_channels))

8640
8640


In [111]:
print(y_encoded.shape)

(64800, 3)


In [112]:
print(len(y_encoded))

64800


In [113]:
new_len = len(y_encoded)//5
y_train = np.array([[0]*3]*new_len)
for i in range(new_len):
    idx= i*5
    select_mid = idx+2 #(select the middle of the 5 4-second epochs as the label)
    y_train[i] = y_encoded[select_mid]
    
print(y_train[0:20])

[[0 1 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]]


In [114]:
print(y_encoded[4:34])

[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [115]:
y_train_new = y_train[0: len_to_keep]
y_cv = y_train[len_to_keep:]
y_train_new = y_train_new[len_channels]
print(len(y_train))
print(len(y_train_new))

12960
8640


In [116]:
# figure out good class weights
print(len(y))
print(len(y[y == 0]))
print(len(y[y == 1]))
print(len(y[y == 2]))
# in percents
class_w0 = len(y)/(len(y[y == 0]))
class_w1 = len(y)/(len(y[y == 1]))
class_w2 = 5
print(class_w0)
print(class_w1)
print(class_w2)
class_weight = {0: class_w0, 1: class_w1, 2: class_w2}

64800
34114
27133
3553
1.8995133962596003
2.388235727711643
5


In [117]:
# train the pre_trained model
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
pre_trained_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy']) 

In [118]:
history = pre_trained_model.fit({"left_signal": train_sh_channels, "mid_signal": train_sh_channels, "right_signal": train_sh_emg}, y_train_new, batch_size = 100, epochs = 10, class_weight = class_weight, validation_data = ([cv_channels, cv_channels, cv_emg], y_cv))



Epoch 1/10
87/87 [==============================] - 19s 216ms/step - loss: 2.5196 - accuracy: 0.5266 - val_loss: 1.0937 - val_accuracy: 0.5100
Epoch 2/10
87/87 [==============================] - 18s 210ms/step - loss: 2.5126 - accuracy: 0.5341 - val_loss: 1.0889 - val_accuracy: 0.5100
Epoch 3/10
87/87 [==============================] - 20s 233ms/step - loss: 2.5058 - accuracy: 0.5341 - val_loss: 1.0844 - val_accuracy: 0.5100
Epoch 4/10
87/87 [==============================] - 20s 234ms/step - loss: 2.4993 - accuracy: 0.5341 - val_loss: 1.0799 - val_accuracy: 0.5100
Epoch 5/10
87/87 [==============================] - 18s 206ms/step - loss: 2.4929 - accuracy: 0.5341 - val_loss: 1.0755 - val_accuracy: 0.5100
Epoch 6/10
87/87 [==============================] - 20s 227ms/step - loss: 2.4867 - accuracy: 0.5341 - val_loss: 1.0711 - val_accuracy: 0.5100
Epoch 7/10
87/87 [==============================] - 18s 206ms/step - loss: 2.4807 - accuracy: 0.5341 - val_loss: 1.0670 - val_accuracy: 0.5100

In [119]:
print(train_sh_channels.shape)

(8640, 2560, 2)


In [120]:
print(train_sh_emg.shape)

(8640, 2432)


In [121]:
# remove the last two layers (flatten, and dense) and replace it with the fine tuning layer
for el in pre_trained_model.layers:
    print(el.name)
pre_trained_model._layers.pop()
pre_trained_model._layers.pop()
print('************************')
for el in pre_trained_model.layers:
    print(el.name)

left_signal
mid_signal
right_signal
conv1d_36
conv1d_40
conv1d_44
max_pooling1d_18
max_pooling1d_20
max_pooling1d_22
dropout_19
dropout_20
dropout_21
conv1d_37
conv1d_41
conv1d_45
conv1d_38
conv1d_42
conv1d_46
conv1d_39
conv1d_43
conv1d_47
max_pooling1d_19
max_pooling1d_21
max_pooling1d_23
concat_axis
flatten_6
dense_13
************************
left_signal
mid_signal
right_signal
conv1d_36
conv1d_40
conv1d_44
max_pooling1d_18
max_pooling1d_20
max_pooling1d_22
dropout_19
dropout_20
dropout_21
conv1d_37
conv1d_41
conv1d_45
conv1d_38
conv1d_42
conv1d_46
conv1d_39
conv1d_43
conv1d_47
max_pooling1d_19
max_pooling1d_21
max_pooling1d_23
concat_axis


In [122]:
#https://stackoverflow.com/questions/41378461/how-to-use-models-from-keras-applications-for-transfer-learnig/41386444#41386444
#new_layer = Dense(10, activation='softmax', name='my_dense')

#inp = model.input
#out = new_layer(model.layers[-1].output)

#model2 = Model(inp, out)

inp = pre_trained_model.input


s = Dropout(0.5)(pre_trained_model.layers[-1].output)

# Left branch (cannot flatten before since LSTM requires input of form (None, x, y) NOT (None, z))
sLeft = Bidirectional(LSTM(units = 512, return_sequences = True, activation='tanh'))(s) #https://stackoverflow.com/questions/40331510/how-to-stack-multiple-lstm-in-keras
sLeft = Bidirectional(LSTM(units = 512, activation='tanh'))(sLeft)
print(sLeft.shape)

# Right branch
q = Flatten()(s)
q = Dense(units = 1024, activation = 'relu')(q)
print(q.shape)

# combine both 1024-length vectors by adding them
q = Add()([q, sLeft])
print(q.shape)

# now dropout 50% and softmax on 3 output neurons
q = Dropout(0.5)(q)
q = Dense(3, activation = 'softmax')(q)

fine_tuned_model = tf.keras.Model(inputs = inp, outputs = q)

(None, 1024)
(None, 1024)
(None, 1024)


In [123]:
# train the pre_trained model
opt2 = tf.keras.optimizers.Adam(learning_rate=1e-6)
fine_tuned_model.compile(optimizer=opt2, loss='categorical_crossentropy', metrics=['accuracy']) 

In [124]:
history2 = fine_tuned_model.fit({"left_signal": train_sh_channels, "mid_signal": train_sh_channels, "right_signal": train_sh_emg}, y_train_new, batch_size = 10, epochs = 20, class_weight = class_weight, validation_data = ([cv_channels, cv_channels, cv_emg], y_cv))



Epoch 1/20
864/864 [==============================] - 430s 498ms/step - loss: 2.5018 - accuracy: 0.5068 - val_loss: 1.0454 - val_accuracy: 0.5100
Epoch 2/20
864/864 [==============================] - 416s 481ms/step - loss: 2.3422 - accuracy: 0.5149 - val_loss: 0.8884 - val_accuracy: 0.5100
Epoch 3/20
864/864 [==============================] - 418s 483ms/step - loss: 2.3027 - accuracy: 0.5139 - val_loss: 0.8837 - val_accuracy: 0.5100
Epoch 4/20
864/864 [==============================] - 418s 484ms/step - loss: 2.3043 - accuracy: 0.5218 - val_loss: 0.8814 - val_accuracy: 0.5100
Epoch 5/20
864/864 [==============================] - 425s 492ms/step - loss: 2.3047 - accuracy: 0.5194 - val_loss: 0.8822 - val_accuracy: 0.5100
Epoch 6/20
864/864 [==============================] - 426s 493ms/step - loss: 2.3011 - accuracy: 0.5189 - val_loss: 0.8882 - val_accuracy: 0.5100
Epoch 7/20
864/864 [==============================] - 432s 499ms/step - loss: 2.3036 - accuracy: 0.5175 - val_loss: 0.8831 -

In [ ]:
lenth = len(eeg1_t)

eeg1_reshaped_t = np.array([[0]*2560]*lenth)
eeg2_reshaped_t = np.array([[0]*2560]*lenth)
emg_reshaped_t = np.array([[0]*2560]*lenth)

eeg1_reshaped_t, eeg2_reshaped_t, emg_reshaped_t = reshape_sig(eeg1_t, eeg2_t, emg_t)
    
rms_vals_t= np.apply_along_axis(rms_transform, axis = 1, arr = emg_reshaped_t)
eegs_comb_t = np.hstack((eeg1_reshaped_t, eeg2_reshaped_t))
print(eegs_comb_t.shape)
def combine_channels(signals):
    sigs = np.split(signals, 2)
    eeg1 = sigs[0]
    eeg2 = sigs[1]
    return np.array((eeg1, eeg2)).T
    
channels_t = np.apply_along_axis(combine_channels, axis = 1, arr = eegs_comb_t)
X_pred = [channels_t, channels_t, rms_vals_t]
prediction = model.predict(X_pred)
dfPredictions = pd.DataFrame(prediction)
dfPredictions.index.name = "id"
dfPredictions.to_csv("task4MCSleepNetPredictions.csv", header = ['y'], index=True)